# Assignment 4 DAT341 - Implementing Linear Classifiers
## Assignment page: https://www.cse.chalmers.se/~richajo/dit866/assignments/a4/assignment4.html#foot1

## Group Members: Mirco Ghadri, Tobias Filmberg, Sameer Jathavedan

### Date: 24-2-2023

# Exercise Question

In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
import pandas as pd

In [2]:
X1 = [{'city':'Gothenburg', 'month':'July'},
      {'city':'Gothenburg', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y1 = ['rain', 'rain', 'sun', 'rain']

X2 = [{'city':'Sydney', 'month':'July'},
      {'city':'Sydney', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y2 = ['rain', 'sun', 'sun', 'rain']

In [3]:
classifier1 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
classifier1.fit(X1, Y1)
guesses1 = classifier1.predict(X1)
print("Training accuracy for X1: ",accuracy_score(Y1, guesses1))

classifier2 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
#classifier2 = make_pipeline(DictVectorizer(), LinearSVC())
classifier2.fit(X2, Y2)
guesses2 = classifier2.predict(X2)
print("Training accuracy for X2: ",accuracy_score(Y2, guesses2))

Training accuracy for X1:  1.0
Training accuracy for X2:  0.5


### Q: Why does the classifier get a 100% training accuracy score for X1 but only a 50% accuracy score for predicting X2?

### The reason the perceptron gets a 100% training accuracy on the first dataset(X1) is because the data is linearly separable. The reason the perceptron only gets a 50% accuracy on the second dataset(X2) is because the data is not linearly separable. It can be shown with a plot that the data in the first dataset is linearly separable and that the data in the second dataset is not linearly separable.

## Dataset n.1: X1 and Y1 - Linearly Separable

The features(X) of the dataset(the month and the city) are in this case plotted on the x and y axis. The output label(Y) is shown with the letter R, which stands for Rainy weather, and the letter S, which stands for Sunny weather. We can see that the data is linearly separable since we can draw a straight line that separates it into 2 homogenous subsets - rainy weather and sunny weather.

<img src="linear_separability.png"/>

## Dataset n.2: X2 and Y2 - Not Linearly Separable

In the case of X2 and Y2, it is not possible for the Perceptron to create a model(line) that separates the output labels(Y) into 2 distinct and homogenous subsets. The best possible accuracy it can get is 50%.

<img src="linear_inseparability.png"/>

In [4]:
import numpy as np
from sklearn.base import BaseEstimator
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Linear Classifier

In [5]:
class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])

### Perceptron

In [6]:
class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                # why does it do <=0. Can the score be equal to 0? The score can only be -1 or 1.
                if y*score <= 0:
                    #subtract the feature vector from the weight vector if the predicted y value was positive but the actual was negative. Add the feature vector to the weight vector if the predicted y value was negative but the real was positive.
                    self.w += y*x

### Perceptron Optional

In [7]:
##### The following part is for the optional task.

### Sparse and dense vectors don't collaborate very well in NumPy/SciPy.
### Here are two utility functions that help us carry out some vector
### operations that we'll need.

def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)


class SparsePerceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm,
    assuming that the input feature matrix X is sparse.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.

        Note that this will only work if X is a sparse matrix, such as the
        output of a scikit-learn vectorizer.
        """
        self.find_classes(Y)

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        Ye = self.encode_outputs(Y)
                        
        # Initialize the weight vector to all zeros.
        self.w = np.zeros(X.shape[1])

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))

        for i in range(self.n_iter):
            for x, y in XY:

                # Compute the output score for this instance.
                # (This corresponds to score = x.dot(self.w) above.)
                score = sparse_dense_dot(x, self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    # (This corresponds to self.w += y*x above.)
                    add_sparse_to_dense(x, self.w, y)

## Document Classifier

In [8]:
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y

In [9]:
# Read all the documents.
X, Y = read_data('all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    #SelectKBest(k=1000),
    Normalizer(),

    # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
    SparsePerceptron()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 2.22 sec.
Accuracy: 0.8300.


## Implementing the SVC

In [14]:
class SVC(LinearClassifier):
    """
    A straightforward implementation of the svc(support vector classifier) learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the svc learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        #the value of the regularizer is 1 divided by the number of instances in the training set, len(Y)
        regularizer = 1/len(Y)
        t = 1
        
        # SVC algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                
                # Compute eta. t starts from 1 because we can not divide by 0
                # the value of eta is large at first, but gradually gets smaller. 
                eta = 1 / (regularizer * t)
                # Compute the output score for this instance.
                score = x.dot(self.w)
                #the bug was caused because we incremented the variable t for each iteration of the outer for loop.
                #the variable t should be incremented for each iteration of the inner for loop.
                t+=1
                # If there was an error, update the weights.
                if y*score < 1:
                    self.w = (1-eta * regularizer)*self.w + (eta * y) * x 
                else:
                    self.w = (1-eta * regularizer)*self.w

In [15]:
# Read all the documents.
X, Y = read_data('all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # SVC classifier that we created
    SVC()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 2.08 sec.
Accuracy: 0.8317.


### We land at an accuracy of 0.8317 when using Pegasos algorithm to create the SVC

### Bonus task 1a) Faster linear algebra operations using BLAS

In [12]:
from scipy.linalg.blas import ddot, dscal, daxpy

In [16]:
class SVC_BLAS(LinearClassifier):
    """
    A straightforward implementation of the svc(support vector classifier) learning algorithm.
    """

    def __init__(self, n_iter=10):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the svc learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        #the value of the regularizer is 1 divided by the number of instances in the training set, len(Y)
        regularizer = 1/len(Y)
        t=1
        
        # SVC algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                eta = 1 / (regularizer * t)
                # Compute the output score for this instance.
                score = ddot(x,self.w)
                
                #the bug in the previous code was caused by the fact that t was incremented in the outer for loop
                #t should be incremented in the inner for loop
                t+=1
                # If there was an error, update the weights.
                if y*score < 1:
                    dscal(1-eta * regularizer, self.w)
                    daxpy(x,self.w,a=eta*y)
                else:
                    dscal(1-eta * regularizer, self.w)

In [17]:
# Read all the documents.
X, Y = read_data('all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # SVC classifier that we created
    SVC_BLAS()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.22 sec.
Accuracy: 0.8326.


The accuracy when running the BLAS functions inside of the SVC is the same: 0.8326.  
However, the computation time has improved to 1.22 seconds compared to 2 seconds previously.

### Bonus task 1b) SparseSVC - an implementation of SVC that deals with sparse training data.

In [31]:
class SparseSVC(LinearClassifier):
    """
    A straightforward implementation of the svc(support vector classifier) learning algorithm.
    """

    def __init__(self, n_iter=10):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the svc learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        #the value of the regularizer is 1 divided by the number of instances in the training set, len(Y)
        regularizer = 1/len(Y)
        t=1
        
        # SVC algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                eta = 1 / (regularizer * t)
                
                # Compute the output score for this instance.
                score = sparse_dense_dot(x,self.w)
                t+=1

                # If there was an error, update the weights.
                if y*score < 1:
                    dscal(1-eta * regularizer, self.w)
                    add_sparse_to_dense(x,self.w,eta*y)
                else:
                    #vector rescaling
                    dscal(1-eta * regularizer, self.w) 

In [32]:
# Read all the documents.
X, Y = read_data('all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    Normalizer(),
    #SelectKBest(k=1000),
    SparseSVC()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 6.04 sec.
Accuracy: 0.8414.


When running the document classifier without SelectKBest(k=1000) and using a simple SVC, the running time is 23.77 seconds. However the accuracy has increased to 0.8410.  
When running the document classifier without SelectKBest(k=1000) but using SparseSVC, we see a significant speed improvement. The training time is now only **6 seconds and the accuracy is also 0.8414**.

The reason for this significant improvement in computation speed when using sparseSVC is that it does not consider the empty parts of the sparse vector(in this case the training data) when it performs addition and multiplication. This allows it to compute its results much faster. For example, when it takes the dot product between the sparse vector x and the dense weight vector, it will not consider the parts in the sparse vector x that are 0 when it computes the dot product.

In order for our sparseSVC implementation to work correctly, we also had to remove the line of code in the SVC that was: 
```if not isinstance(X, np.ndarray):
    X = X.toarray()```    

The reason is that we want the training data to be in a sparse matrix format and not a regular numpy array. So we do not want to convert it to a regular numpy matrix.

### Bonus task 1c) SparseSVCOptimized - an optimized implementation of SparseSVC

There is yet another way to speed up the training time. This method is described in section 4 of the clarification document. Instead of scaling the weight vector directly, we accumulate the scaling factors and scale the vector at the end. This is because scaling the vector for each iteration is cost-heavy, especially if it is a large vector. So we only scale the vector 1 time at the end.

In [33]:
class SparseSVCOptimized(LinearClassifier):
    """
    A straightforward implementation of the svc(support vector classifier) learning algorithm.
    """

    def __init__(self, n_iter=10):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the svc learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        #the value of the regularizer is 1 divided by the number of instances in the training set, len(Y)
        regularizer = 1/len(Y)
        a=1
        #we set to 2, because if we set it to 1, a will become 0 and division by 0 is not allowed
        t=2
        # SVC algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                eta = 1 / (regularizer * t)
                #instead of scaling self.w and then multiplying it by x, simply multiply the dot product by the scaling factor
                score = a*sparse_dense_dot(x,self.w)
                a = (1-eta*regularizer)*a     
                t+=1
                # If there was an error, update the weights.
                if y*score < 1:
                    add_sparse_to_dense(x,self.w,eta*y/a)
        self.w = self.w*a

In [34]:
# Read all the documents.
X, Y = read_data('all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    Normalizer(),
    #SelectKBest(k=1000),
    SparseSVCOptimized()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 5.08 sec.
Accuracy: 0.8397.


After optimizing the SparseSVC implementation, we gain 1 second of faster training without SelectKBest(k=1000).
Final result:
Training time: **5 sec**.
Accuracy: **0.8397**.

## Logistic Regression

In [35]:
class LR(LinearClassifier):
    """
    A straightforward implementation of the LR(logistic regression) learning algorithm.
    """
    def __init__(self, n_iter=10):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the logistic regression learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        #the value of the regularizer is 1 divided by the number of instances in the training set, len(Y)
        regularizer = 1/len(Y)
        t=1
        
        # SVC algorithm:
        for t in range(1,self.n_iter+1):
            for x, y in zip(X, Ye):
                eta = 1 / (regularizer * t)
                # Compute the output score for this instance.
                score = x.dot(self.w)
                t+=1
                log_loss_gradient = (-y/(1+np.exp(y*score)))*x
                gradient = regularizer * self.w + log_loss_gradient
                # If there was an error, update the weights.
                self.w = self.w - eta * gradient

In [36]:
# Read all the documents.
X, Y = read_data('all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # SVC classifier that we created
    LR()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 2.01 sec.
Accuracy: 0.8229.
